# Proyecto final de Bases de Datos Avanzadas
## Implementación y Consultas en MongoDB
### 30 de abril de 2025
---

#### Configuración y Carga de Datos

Iniciamos un entorno virtual de python y bajamos las dependencias necesarias (`pymongo` para interactuar con MongoDB y `pandas` para facilitar la lectura del JSON):

In [ ]:
# Si ya tienes las dependencias instaladas, puedes omitir esta celda
# !pip install pymongo pandas
import sys
!{sys.executable} -m pip install pymongo pandas

Importamos las librerías necesarias:

In [ ]:
import json
from pymongo import MongoClient
import pandas as pd
import datetime

Cargamos los datos del archivo `books.json`. Dado que es un archivo JSON donde cada línea es un documento separado, usamos `pandas.read_json` con `lines=True`.

In [ ]:
# Cargar el JSON línea por línea
# df = pd.read_json('books.json', lines=True)
df = pd.read_json('books.json', lines=True)

Convertimos el DataFrame a una lista de diccionarios, que es un formato ideal para insertar en MongoDB.

In [ ]:
# Convertir DataFrame a lista de diccionarios para pymongo
# Aseguramos que los valores NaN se manejen correctamente
data = df.where(pd.notna(df), None).to_dict('records')

Verificamos que se hayan cargado los docuemntos del archivo JSON y mostramos un ejemplo.

In [ ]:
print(f"Se cargaron {len(data)} documentos del archivo JSON.")
# Mostrar un ejemplo de la estructura del primer documento cargado
if data:
    import pprint
    print("\nEjemplo de un documento cargado:")
    pprint.pprint(data[0])

#### Conexión a MongoDB e Inserción de Datos

Establecemos la conexión con el servidor de MongoDB (asegúrate de que tu servidor esté corriendo en `localhost:27017` o ajusta la URI de conexión según sea necesario). Nos conectamos a la base de datos `proyecto_final` y seleccionamos la colección `books`.

In [ ]:
MONGO_URI = 'mongodb://localhost:27017/' 
DATABASE_NAME = 'proyecto_final'
COLLECTION_NAME = 'books'

try:
    client = MongoClient(MONGO_URI)
    db = client[DATABASE_NAME]
    collection = db[COLLECTION_NAME]
    
    # Verificar conexión (opcional)
    client.admin.command('ping')
    print(f"Conexión a MongoDB exitosa. Usando base de datos '{DATABASE_NAME}'.")

except Exception as e:
    print(f"Error al conectar a MongoDB: {e}")
    client = None # Asegurarse de que client sea None si la conexión falla

Antes de insertar, limpiamos la colección para evitar duplicados si ejecutamos el notebook varias veces. Luego insertamos todos los documentos cargados del JSON.

In [ ]:
if client: # Procede solo si la conexión fue exitosa
    try:
        # Eliminar documentos existentes en la colección
        delete_result = collection.delete_many({})
        print(f"Documentos existentes eliminados: {delete_result.deleted_count}")

        # Insertar los nuevos documentos
        # pymongo maneja automáticamente los tipos de datos como fechas, arrays, etc.
        if data:
            insert_result = collection.insert_many(data)
            print(f"Documentos insertados: {len(insert_result.inserted_ids)}")
        else:
            print("No hay datos para insertar.")

    except Exception as e:
        print(f"Error durante la inserción de datos: {e}")
else:
    print("No se pudo establecer conexión con MongoDB. Saltando inserción.")

Verificamos la cantidad de documentos insertados en la colección `books`.

In [ ]:
if client:
    try:
        count = collection.count_documents({})
        print(f"Total de documentos en la colección '{COLLECTION_NAME}': {count}")
        
        print("\nPrimeros 5 documentos en la colección:")
        # Usamos projection para mostrar solo algunos campos para brevedad
        sample_docs = list(collection.find({}, {'_id': 1, 'title': 1, 'isbn': 1}).limit(5))
        for doc in sample_docs:
            print(doc)
            
    except Exception as e:
        print(f"Error durante la verificación de documentos: {e}")
else:
    print("Conexión a MongoDB no establecida. Saltando verificación.")

#### Consultas a la Colección

##### 1. Consulta por título: Buscar libros por su título.

In [ ]:
if client:
    search_title = "Action"
    # Usamos $regex para buscar títulos que contengan la palabra, i para case-insensitive
    query = {"title": {"$regex": search_title, "$options": "i"}}
    
    print(f"Buscando libros con '{{search_title}}' en el título...")
    results = list(collection.find(query, {'_id': 1, 'title': 1, 'authors': 1}).limit(10)) # Limitar resultados para mostrar

    df_results = pd.DataFrame(results)
    print(f"Encontrados {len(results)} resultados (mostrando los primeros 10):")
    display(df_results) # Usar display para mostrar DataFrame en Jupyter

##### 2. Consulta por ISBN: Buscar un libro específico mediante su número ISBN.

In [ ]:
if client:
    search_isbn = "1933988673"
    query = {"isbn": search_isbn}
    
    print(f"Buscando libro con ISBN '{search_isbn}'...")
    results = list(collection.find(query, {'_id': 1, 'title': 1, 'isbn': 1}))

    df_results = pd.DataFrame(results)
    print(f"Encontrados {len(results)} resultados:")
    display(df_results)

##### 3. Consulta por autor: Obtener todos los libros escritos por un autor en particular.

In [ ]:
if client:
    search_author = "W. Frank Ableson"
    # Buscamos documentos donde el array 'authors' contenga el nombre del autor
    query = {"authors": search_author}
    
    print(f"Buscando libros del autor '{search_author}'...")
    results = list(collection.find(query, {'_id': 1, 'title': 1, 'authors': 1}))

    df_results = pd.DataFrame(results)
    print(f"Encontrados {len(results)} resultados:")
    display(df_results)

##### 4. Consulta por categoría: Listar libros que pertenecen a una categoría específica.

In [ ]:
if client:
    search_category = "Java"
    # Buscamos documentos donde el array 'categories' contenga la categoría
    query = {"categories": search_category}
    
    print(f"Buscando libros en la categoría '{search_category}'...")
    results = list(collection.find(query, {'_id': 1, 'title': 1, 'categories': 1}))

    df_results = pd.DataFrame(results)
    print(f"Encontrados {len(results)} resultados:")
    display(df_results)

##### 5. Consulta por estado: Filtrar libros según su estado.

In [ ]:
if client:
    search_status = "PUBLISH"
    # Basado en la estructura del JSON de ejemplo, 'status' parece ser un string, no un arreglo
    query = {"status": search_status}
    
    print(f"Buscando libros con estado '{search_status}'...")
    results = list(collection.find(query, {'_id': 1, 'title': 1, 'status': 1}))

    df_results = pd.DataFrame(results)
    print(f"Encontrados {len(results)} resultados:")
    display(df_results)

##### 6. Número total de libros: Contar el total de libros almacenados en la colección.

In [ ]:
if client:
    count = collection.count_documents({})
    print(f"El número total de libros en la colección es: {count}")

##### 7. Buscar libros publicados después de una fecha específica

In [ ]:
if client:
    # Convertir la fecha a un objeto datetime para la consulta
    search_date = datetime.datetime(2010, 1, 1)
    # Usamos $gte (greater than or equal) para encontrar fechas posteriores o iguales
    # La estructura de fecha en el JSON original es un objeto con '$date'
    query = {"publishedDate": {"$gte": search_date}}
    
    print(f"Buscando libros publicados después de {search_date.date()}...")
    # Limitar resultados para mostrar
    results = list(collection.find(query, {'_id': 1, 'title': 1, 'publishedDate': 1}).limit(10))

    df_results = pd.DataFrame(results)
    print(f"Encontrados {len(results)} resultados (mostrando los primeros 10):")
    display(df_results)

##### 8. Encontrar libros con un número de páginas dentro de un rango

In [ ]:
if client:
    min_pages = 400
    max_pages = 600
    # Usamos $gte (greater than or equal) y $lte (less than or equal) para el rango
    query = {"pageCount": {"$gte": min_pages, "$lte": max_pages}}
    
    print(f"Buscando libros con un número de páginas entre {min_pages} y {max_pages}...")
     # Limitar resultados para mostrar
    results = list(collection.find(query, {'_id': 1, 'title': 1, 'pageCount': 1}).limit(10))

    df_results = pd.DataFrame(results)
    print(f"Encontrados {len(results)} resultados (mostrando los primeros 10):")
    display(df_results)

##### 9. Listar autores distintos presentes en la base de datos

In [ ]:
if client:
    # Usamos distinct para obtener los valores únicos del campo 'authors'
    # Dado que 'authors' es un array, distinct nos dará cada nombre de autor único en todos los arrays.
    try:
        distinct_authors = collection.distinct("authors")
        
        print(f"Total de autores distintos encontrados: {len(distinct_authors)}")
        print("\nPrimeros 20 autores distintos:")
        # Mostrar solo los primeros 20 para brevedad
        for i, author in enumerate(distinct_authors[:20]):
            print(f"- {author}")
    except Exception as e:
        print(f"Error al obtener autores distintos: {e}")

##### 10. Contar libros por estado

In [ ]:
if client:
    # Usamos el framework de agregación para agrupar por estado y contar
    pipeline = [
        {"$group": {"_id": "$status", "count": {"$sum": 1}}}
    ]
    
    print("Contando libros por estado...")
    results = list(collection.aggregate(pipeline))

    df_results = pd.DataFrame(results)
    # Renombrar columnas para claridad
    df_results.rename(columns={'_id': 'estado', 'count': 'cantidad'}, inplace=True)
    display(df_results)

##### 11. Encontrar libros que tienen una descripción corta pero no una larga

In [ ]:
if client:
    # Buscamos documentos donde shortDescription exista y longDescription sea null o no exista
    # $exists: True -> el campo existe
    # $exists: False -> el campo no existe
    # O podemos usar $in: [None] para buscar null (si es el caso)
    # O verificar si es null usando {"$type": 10} para null BSON type
    # Basado en el JSON, la ausencia del campo o su valor None son posibilidades
    # Usaremos $exists y un check para None/null

    # Consulta para documentos con shortDescription que existe Y longDescription que es null O no existe
    query = {
        "shortDescription": {"$exists": True, "$ne": None}, # Aseguramos que shortDescription existe y no es null
        "$or": [
            {"longDescription": None}, # longDescription es explicitamente null
            {"longDescription": {"$exists": False}} # longDescription no existe
        ]
    }
    
    print("Buscando libros con descripción corta pero sin descripción larga...")
    # Limitar resultados para mostrar
    results = list(collection.find(query, {'_id': 1, 'title': 1, 'shortDescription': 1, 'longDescription': 1}).limit(10))

    df_results = pd.DataFrame(results)
    print(f"Encontrados {len(results)} resultados (mostrando los primeros 10):")
    display(df_results)

Vemos qué libros ha escrito "Robi Sen":

In [ ]:
author = "Robi Sen"

# La consulta busca documentos donde el array 'authors' contenga el valor 'author'
query = {"authors": author}

print(f"Buscando libros escritos por '{author}'...")

# Ejecutamos la consulta y seleccionamos solo los campos _id, title, y authors para mostrar
results_cursor = collection.find(query, {'_id': 1, 'title': 1, 'authors': 1})

# Convertir el cursor a una lista y luego a DataFrame para visualización
results_list = list(results_cursor)
df_results = pd.DataFrame(results_list)

print(f"Encontrados {len(results_list)} resultados:")
display(df_results)